In [ ]:
using DynamicProgramming, JSON
include("ecow.jl")

using Plots
const mm = Plots.mm
fntsm = Plots.font("times", 12)
fntlg = Plots.font("times", 14)
default(
    titlefont=fntlg,
    guidefont=fntlg,
    tickfont=fntsm,
    legendfont=fntsm,
    left_margin=3mm,
    bottom_margin=10mm,
    top_margin=0mm,
    right_margin=0mm,
    size=(500,300),
    xlabel="Days of the Season\n",
    label="",
    w=3,
    c="#00467F"
)
gr()

In [ ]:
f(q) = min(2.0, q/7.5)
g(ρ, Q) = ρ * f(Q/ρ - 2.75)

Ρ = 0:0.1:3.0
Q = 0:1:60
M = [
    max(0.0, g(ρ, q)) for ρ in Ρ, q in Q
]
contour(Q, Ρ, M,
    levels = 20,
    title  = "Milk Produced (kg/Ha/day)",
    xlabel = "Q (kg/Ha/day)\n",
    ylabel = "Stocking Rate (Cows/Ha)"
)
savefig("moo_intuition.pdf")

In [ ]:
cow = ECow.Cow()
herbage = ECow.FoodOffer(ECow.Pasture(), 35.0)
supplement = ECow.FoodOffer(ECow.PKE(), 2.0)

milkingplan(t,n=365) = vcat(trues(t), falses(n-t))

results = Dict(
    :bcs         => vcat(3.2, zeros(365)),
    :maintenance => vcat(63, zeros(365)),
    :lactation   => milkingplan(40 * 7),  # milk for 40 weeks
    :supplement  => zeros(365),
    :herbage     => zeros(365),
    :milksolids  => zeros(365)
)

for day in 1:365
    (new_bcs, new_maintenance, actual_herbage_intake, actual_supplement_intake, milksolids) = ECow.updateday!(
        cow, day, results[:bcs][day], results[:maintenance][day], results[:lactation][day], supplement, herbage
    )
    results[:bcs][day+1]         = new_bcs
    results[:maintenance][day+1] = new_maintenance
    results[:supplement][day]    = actual_supplement_intake
    results[:herbage][day]       = actual_herbage_intake
    results[:milksolids][day]    = milksolids
end

bcs_plot = plot(results[:bcs],
    ylabel="BCS (Units)",
    title="(a)",
    ylims=(0,5)
)

mai_plot = plot(results[:maintenance],
    ylabel="Maintenance (MJ/Day)",
    title="(b)",
    ylims=(50,80)
)

her_plot = plot(
    ylabel="Feed Intake (kg/Day)",
    title="(c)",
    ylims=(0,14),
    legend=:topleft
)
plot!(results[:herbage], label="Herbage")
plot!(results[:supplement], label="Supplement", linestyle=:dash)

mlk_plot = plot(results[:milksolids],
    ylabel="Milk Solids (kg/Day)",
    title="(d)",
    ylims=(0,1.5)
)

plot(bcs_plot, mai_plot, her_plot, mlk_plot, size=(1000, 600))

savefig("moo_example.pdf")

In [ ]:
PARAMS, RESULTS, SOLVED_MODEL = "moo.params", "moo.results", "moo.solved.model"

summary = JSON.parsefile(RESULTS)
params = JSON.parsefile(PARAMS)

# ==========================================
# back out the pasture cover
cover = zeros(length(summary["herbage"]))
i = 1
current_cover = 2500.0
for week in 1:52
    for day in 1:7
        current_cover += params["grass_growth"][week] -
            params["stocking_rate"] * summary["herbage"][i]
        cover[i] = current_cover
        i += 1
    end
end
# ==========================================

#=
    Plot simulated policy
=#
feed_plot = plot(
    title="(e)", ylabel="Quantity (kg/cow/day)",
    ylims=(0,15),
    legend=:topleft
)
plot!(summary["herbage"], label="Herbage")
plot!(summary["supplement"], linestyle=:dash, label="Supplement")
plot(
    plot(summary["bcs"],
        title="(a)", ylabel="BCS (Units)",
        ylims=(0, 5)
    ),
    plot(summary["maintenance"],
        title="(b)", ylabel="Maintenance (MJ/day)",
        ylims=(50, 80)
    ),
    plot(summary["lactation"],
        title="(c)", ylabel="Lactation",
        ylims=(0, 1)
    ),
    plot(cover,
        title="(d)", ylabel="Pasture Cover (kg/Ha)",
        ylims=(0, 3500)
    ),
    feed_plot,
    plot(summary["milksolids"],
        title="(g)", ylabel="Milk Solids (kg/cow/day)",
        ylims=(0, 2)
    ),
    size=(1500,600)
)
savefig("moo_simulation.pdf")

#=
    Plot the value to go.
=#
m = open(SOLVED_MODEL, "r") do io
   deserialize(io) 
end

function plotvaluetogo(stage, title="")
    X = 2.5:0.1:4.5
    Y = 2000:50:3000
    stage_one_surface = [
        -m.stages[stage].interpolatedsurface[bcs,70.0,1.0,cover]
        for bcs in X, cover in Y
    ]
    contour(X, Y, stage_one_surface',
        title        = title,
        xlabel       = "BCS\n",
        ylabel       = "Pasture Cover (kg/Ha)",
        right_margin = 7.5mm,
        left_margin  = 5mm,
        levels       = 20
    )
end
plot(
    plotvaluetogo(1, "(a) Week 1"),
    plotvaluetogo(25, "(b) Week 25"),
    plotvaluetogo(50, "(c) Week 50"),
    size=(1500, 300), layout=(1,3)
)
savefig("moo_value-to-go.pdf")

# summarize 
println("Lactation Length:  ",  sum(summary["lactation"]) / 7)
println("Milksolids kg/Ha:  ",  round(Int, summary["total_milksolids"] * 3))
println("Milksolids kg/Cow: ",  round(Int, summary["total_milksolids"]))
println("Milksolids \$/Ha:   ", round(Int, summary["total_milksolids"] * 6* 3))
println("Pasture kg/Ha:     ",  round(0.001 * summary["total_herbage"] * 3, 2))
println("Palm Kernel kg/Ha: ",  round(0.001 * summary["total_supplement"] * 3, 2))
println("Feed Imported %:   ",  round(100 * summary["total_supplement"] / (summary["total_herbage"] + summary["total_supplement"]), 1))
println("Palm Kernel \$/Ha:  ", round(Int, summary["total_supplement"] * 0.5 * 3))
println("FEI Penalty \$/Ha:  ", round(Int, sum(0.25(summary["supplement"] - 3)) * 3))

In [ ]:
Q = 0:100:10_000
Ρ = 2.5:0.25:4.0

M = zeros(length(Ρ), length(Q))
S = zeros(length(Ρ), length(Q))

for (k,q) in enumerate(Q)
    for (j,ρ) in enumerate(Ρ)    
        d = JSON.parsefile("results/$(ρ)_$(q)_moo.results")
        M[j,k] = ρ * sum(d["milksolids"])
        S[j,k] = ρ * sum(d["supplement"])
    end
end

contour(Q, Ρ, M,
    xlabel = "Total Quantity of Palm Kernel (kg/Ha)\n",
    ylabel = "Stocking Rate (Cows/Ha)"
)
savefig("moo_total_milksolids.pdf")

In [ ]:
Q = 0:100:10_000
Ρ = 2.5:0.25:4.0
M = [6.1,7.6,6.08,5.84,8.4,4.4,3.9]

Π = zeros(length(M), length(Ρ), length(Q))

for (k,q) in enumerate(Q)
    for (j,ρ) in enumerate(Ρ)    
        d = JSON.parsefile("results/$(ρ)_$(q)_moo.results")
        for (i,m) in enumerate(M)
            Π[i,j,k] = ρ * (m * sum(d["milksolids"]) - 0.5 * sum(d["supplement"]) - 589) - 1467
        end
    end
end

# wait-and-see
wait_and_see = [
    maximum(Π[i, :, :])
    for (i,m) in enumerate(M)
]
println("Wait-and-See: ", round(mean(wait_and_see), 2),"  ", round(std(wait_and_see), 2))

# here and now solution
expected_profit = [
    mean(Π[:,j,k]) for j in 1:length(Ρ), k in 1:length(Q)
]
(ρi, qi) = ind2sub(expected_profit, indmax(expected_profit))
here_and_now = Π[:, pi, qi]
println("Here-and-Now: ", round(mean(here_and_now), 2),"  ", round(std(here_and_now), 2))
println("EVPI = ", mean(wait_and_see) - mean(here_and_now))

utility(x, alpha=1) = 1 - exp(-alpha * x / 1000)

function computeriskaversesolution(alpha, Π, Ρ, Q)
    alpha = 1.0
    expected_profit_risk = [
        mean(utility.(Π[:,j,k], alpha)) for j in 1:length(Ρ), k in 1:length(Q)
    ]
    (ρi, qi) = ind2sub(expected_profit_risk, indmax(expected_profit_risk))
    here_and_now_risk = Π[:, pi, qi]
    Ρ[ρi], Q[qi], mean(here_and_now_risk), mean(here_and_now_risk), minimum(here_and_now_risk)
end

expected_profit = [
    mean(Π[:,j,k]) for j in 1:length(Ρ), k in 1:length(Q)
]
std_profit = [
    std(Π[:,j,k]) for j in 1:length(Ρ), k in 1:length(Q)
]

mean_plot = contour(Q, Ρ, expected_profit,
    title  = "(a)",
    xlabel = "Total Quantity of Palm Kernel (kg/Ha)\n",
    ylabel = "Stocking Rate (Cows/Ha)",
    right_margin=5mm
)
std_plot = contour(Q, Ρ, std_profit,
    title  = "(b)",
    xlabel = "Total Quantity of Palm Kernel (kg/Ha)\n",
    ylabel = "Stocking Rate (Cows/Ha)",
    right_margin=5mm
)

for α in [1/1, 1/2, 1/3, 1/4]
    p, q, μ, σ, mn = computeriskaversesolution(α, Π, Ρ, Q)
    println("α = $(round(α, 2)), μ=$(round(μ, 2)), σ=$(round(σ, 2)), worst=$(round(mn, 2))")
    scatter!(mean_plot, [q], [p])
    scatter!(std_plot, [q], [p])
end

plot(mean_plot, std_plot, size=(1000, 300))
savefig("moo_risk.pdf")

In [ ]:
summary = JSON.parsefile("results/2.5_0_moo.results")
params = JSON.parsefile(PARAMS)

# ==========================================
# back out the pasture cover
cover = zeros(length(summary["herbage"]))
i = 1
current_cover = 2500.0
for week in 1:52
    for day in 1:7
        current_cover += params["grass_growth"][week] -
            params["stocking_rate"] * summary["herbage"][i]
        cover[i] = current_cover
        i += 1
    end
end
# ==========================================

#=
    Plot simulated policy
=#
feed_plot = plot(
    title="(e)", ylabel="Quantity (kg/cow/day)",
    ylims=(0,15),
    legend=:topleft
)
plot!(summary["herbage"], label="Herbage")
plot!(summary["supplement"], linestyle=:dash, label="Supplement")
plot(
    plot(summary["bcs"],
        title="(a)", ylabel="BCS (Units)",
        ylims=(0, 5)
    ),
    plot(summary["maintenance"],
        title="(b)", ylabel="Maintenance (MJ/day)",
        ylims=(50, 80)
    ),
    plot(summary["lactation"],
        title="(c)", ylabel="Lactation",
        ylims=(0, 1)
    ),
    plot(cover,
        title="(d)", ylabel="Pasture Cover (kg/Ha)",
        ylims=(0, 3500)
    ),
    feed_plot,
    plot(summary["milksolids"],
        title="(g)", ylabel="Milk Solids (kg/cow/day)",
        ylims=(0, 2)
    ),
    size=(1500,600)
)
savefig("moo_simulation.pdf")